In [17]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd
import numpy as np
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import clean as clean
from sklearn.utils import resample

In [19]:
# Helper function to calculate False Negative Rate
def evaluate_fnr(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1, 0]
    tp = cm[1, 1]
    return fn / (fn + tp)

In [20]:
# Load the raw data
dt_raw = pd.read_csv('../data/TBI PUD 10-08-2013.csv')
dt_clean = clean.clean_data('../data/TBI PUD 10-08-2013.csv', remove_feats_after_ct=True, remove_TBI_rows_with_nan=True,threshold=0.5,rm_feats=True,remove_GCS_total_mismatch=True)

In [21]:
# train and test load
X_train_balanced = np.load('../train_test/X_train_balanced.npy')
y_train_balanced = np.load('../train_test/y_train_balanced.npy')
X_test = np.load('../train_test/X_test.npy')
y_test = np.load('../train_test/y_test.npy')

In [22]:
# QDA
# Initialize model
qda_model = QuadraticDiscriminantAnalysis()
qda_model.fit(X_train_balanced, y_train_balanced)
# Predictions for QDA
y_pred_qda = qda_model.predict(X_test)
y_proba_qda = qda_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
# save the model in results folder
np.save('../results/qda.npy', y_proba_qda)

/Users/vidhisinha/Downloads/ENTER/envs/215a/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [23]:
# False negative rate for LDA
fnr_qda = evaluate_fnr(y_test, y_pred_qda)
print(f'False Negative Rate for LDA: {fnr_qda}')

False Negative Rate for LDA: 0.2987012987012987


In [24]:
# Evaluation Metrics for LDA
print("=== LDA Evaluation ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_qda))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_qda))
print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba_qda))

=== LDA Evaluation ===
Confusion Matrix:
[[7644  610]
 [  46  108]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.93      0.96      8254
         1.0       0.15      0.70      0.25       154

    accuracy                           0.92      8408
   macro avg       0.57      0.81      0.60      8408
weighted avg       0.98      0.92      0.95      8408


ROC AUC Score:
0.9202527542726235
